In [5]:
import spacy
from spacy.tokens import Doc
import re

In [6]:
# input_string = 'Diversity in workforce is important. Continental is aware that legal and cultural requirements can vary in a global market. We expect all of our suppliers to be guided by fairness, honesty and responsibility in all aspects of their business. Our supplier code establishes important standards that match the Continental corporate values. Every supplier must comply strictly with these standards. We use them to define requirements for good working conditions, then check compliance with these requirements through our on-site audits.'
# input_string = 'At Bosch, diversity is a fundamental pillar. The company intends to increase workforce diversity by 40%'

input_string = 'As a global financial institution, Deutsche Bank operates in various countries, each of which imposes its own regulations (often with extra- territorial implications). These define how we operate, as well as our conduct, behavior, and standards to which we must adhere. Our strategy and execution model is affected by different political environments and a large number of regulatory requirements. We remain continually aware of these forces that influence our business, and we engage in political and regulatory decisions. This is fundamental to understanding wider political developments and the evolution of the regulatory environment, as well as fostering stakeholder trust. In recent times, international and national political systems have shown signs of fragmentation. This directly affects our business model. In 2017 alone, we saw crucial elections in France, the UK, and Germany, as well as a new US government whose Con- gress passed a comprehensive tax reform that had an immediate impact on our US tax position. The UK has formally declared its exit from the EU, and negotiations are under way. This will have repercussions on our structure, operations, client relationships, and staffing. Furthermore, wider political developments in the Euro zone (such as important national elections) will impact the stability of financial markets, market prices, and long-term investment decisions by companies. All of this affects our entire value chain.'

coref_model = spacy.load('en_coreference_web_trf')
doc = coref_model(input_string) 

def resolve_references(doc):
    token_mention_mapper = {}
    output_string = ''
    clusters = [val for key, val in doc.spans.items() if key.startswith('coref_cluster')]

    for cluster in clusters:
        first_mention = cluster[0]

        for mention_span in list(cluster)[1:]:
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
            
            for token in mention_span[1:]:
                token_mention_mapper[token.idx] = ''

    for token in doc:
        
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        else:
            output_string += token.text + token.whitespace_

    return output_string

resolved_string = resolve_references(doc)
print(resolved_string)

As a global financial institution, Deutsche Bank operates in various countries, each of which imposes each of which imposes its own regulations own regulations (often with extra- territorial implications). its own regulations define how Deutsche Bank operate, as well as Deutsche Bank conduct, behavior, and standards to which Deutsche Bank must adhere. Deutsche Bank strategy and execution model is affected by different political environments and a large number of regulatory requirements. Deutsche Bank remain continually aware of these forces that influence Deutsche Bank business, and Deutsche Bank engage in political and regulatory decisions. This is fundamental to understanding wider political developments and the evolution of the regulatory environment, as well as fostering stakeholder trust. In recent times, international and national political systems have shown signs of fragmentation. shown directly affects Deutsche Bank business model. In 2017 alone, Deutsche Bank saw crucial elec

In [7]:
nlp = spacy.load('en_core_web_trf')
ner_op_string = resolved_string

def show_ents(doc):
    global ner_op_string
    
    if doc.ents:
        
        for ent in doc.ents:
            # print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_)))
            
            if ent.label_ == 'ORG':
                word_length = ent.end_char - ent.start_char
                ner_op_string = ner_op_string[:ent.start_char] + 'O'*(word_length) + ner_op_string[ent.end_char:]
    
    else:
        print('No named entities found.')
    
    return ner_op_string
    
doc = nlp(resolved_string)
show_ents(doc)
ner_op_string = re.sub(r'O{2,}', 'ORG',ner_op_string)
print(ner_op_string)

As a global financial institution, ORG operates in various countries, each of which imposes each of which imposes its own regulations own regulations (often with extra- territorial implications). its own regulations define how ORG operate, as well as ORG conduct, behavior, and standards to which ORG must adhere. ORG strategy and execution model is affected by different political environments and a large number of regulatory requirements. ORG remain continually aware of these forces that influence ORG business, and ORG engage in political and regulatory decisions. This is fundamental to understanding wider political developments and the evolution of the regulatory environment, as well as fostering stakeholder trust. In recent times, international and national political systems have shown signs of fragmentation. shown directly affects ORG business model. In 2017 alone, ORG saw crucial elections in France, the UK, and Germany, as well as a new US government whose Con- gress passed a compr

In [8]:
def get_org_sentences(ner_op_string):
    temp_list = ner_op_string.split('. ')
    return '. '.join(list(filter(lambda x: 'ORG' in x, temp_list)))

print(get_org_sentences(ner_op_string))

As a global financial institution, ORG operates in various countries, each of which imposes each of which imposes its own regulations own regulations (often with extra- territorial implications). its own regulations define how ORG operate, as well as ORG conduct, behavior, and standards to which ORG must adhere. ORG strategy and execution model is affected by different political environments and a large number of regulatory requirements. ORG remain continually aware of these forces that influence ORG business, and ORG engage in political and regulatory decisions. shown directly affects ORG business model. In 2017 alone, ORG saw crucial elections in France, the UK, and Germany, as well as a new US government whose Con- gress passed a comprehensive tax reform that had an immediate impact on ORG US tax position. the UK has formally declared the UK exit from the ORG, and negotiations are under way. declared will have repercussions on ORG structure, operations, client relationships, and sta